# **SML Project: Binary Tree Predictors**

*   **Author:** Matteo Onger
*   **Date:** September 2024

**Dataset documentation**:
*   [Secondary Mushroom](https://archive.ics.uci.edu/dataset/848/secondary+mushroom+dataset)

In [ ]:
!pip install ucimlrepo

In [15]:
from ucimlrepo import fetch_ucirepo

secondary_mushroom = fetch_ucirepo(id=848)